Author: Nathaniel Blair-Stahn<br>
Date: July 30, 2019<br>
Purpose: Verify bug fix for ylds in Nigeria output and test `sqlns_output_processing.py`

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
# from ipywidgets import interact, IntSlider

pd.set_option('display.max_rows', 8)

!date
!whoami

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import vivarium_conic_sqlns.verification_and_validation.sqlns_output_processing_orig as sop

## Load output from new run

In [ ]:
result_dir = '/share/costeffectiveness/results/sqlns/presentation/nigeria/2019_07_30_00_01_45'

In [ ]:
output = sop.load_output(result_dir, 'output.hdf')
output

In [ ]:
output.filter(like='sqlns')

## Prototype code for dropping draws that are missing random seeds

When I first ran this on 7/31, there were about 35 jobs missing (out of 9720), and 15 out of the 18 draws were missing a seed in at least one scenario. So this would have dropped all but 3 of the draws.

I used this code to help Yongquan subset the output to complete draws, and to help myself write the `drop_incomplete_draws` function in `2019_07_31_icers_all_locations.ipynb`

In [ ]:
# 540 = 108*5 = scenarios*seeds
output.groupby('input_draw').random_seed.count()[output.groupby('input_draw').random_seed.count() != 540]

In [ ]:
output.loc[55]

In [ ]:
output.input_draw.unique()

In [ ]:
bad_draws = [55, 223, 232]
~output['input_draw'].isin(bad_draws)

## Check whether 'treated_days' column got fixed - yes, it's now positive

In [ ]:
output.filter(like='treated_days')

## Compare total ylds to sum of ylds over all columns and to ylds due to iron deficiency

In [ ]:
def check_yld_sum(output):
    """
    Computes the difference between sum of ylds for all causes and total ylds column.
    The difference should be positive for each row of output.
    """
    differences = output.filter(regex='ylds').sum(axis=1) - output.years_lived_with_disability
    print(differences.describe())
    differences.hist(bins=20)

In [ ]:
check_yld_sum(output)

In [ ]:
def check_iron_ylds(output):
    """
    Computes the difference between total ylds column and ylds due to iron deficiency.
    The difference should be positive for each row of output.
    """
    differences = output.years_lived_with_disability - output.ylds_due_to_iron_deficiency
    print(differences.describe())
    differences.hist(bins=20)

In [ ]:
check_iron_ylds(output)

## Check the values of total ylds, ylds due to iron deficiency, and ylds due to PEM

It looks like things are in order. Iron deficiency is still the vast majority of the ylds

These values are for each random seed. Since there are 5 seeds per draw, overall values for each draw should be about 5 times as big.

In [ ]:
print(output.years_lived_with_disability.describe())
output.years_lived_with_disability.hist()

In [ ]:
print(output.ylds_due_to_iron_deficiency.describe())
output.ylds_due_to_iron_deficiency.hist()

In [ ]:
print(output.ylds_due_to_protein_energy_malnutrition.describe())
output.ylds_due_to_protein_energy_malnutrition.hist()

## Check code in new module `sqlns_output_processing.py`

Woo hoo! It seems to work.

In [ ]:
df = sop.get_transformed_data(output)
df

In [ ]:
averted_df = sop.get_averted_results(df)
averted_df

In [ ]:
aggregated_df = sop.get_final_table(averted_df)
aggregated_df